In [1]:
%matplotlib widget
import numpy as np
import math
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import random

import matplotlib.pyplot as plt

import re

from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

from joblib import Parallel, delayed

import multiprocessing
from datetime import datetime

from torch.utils.data import DataLoader, Dataset
from IPython.display import clear_output

In [2]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [3]:
torch.manual_seed(1010101011)
random.seed(1010101011)

In [4]:
ftrainset = open("../dataset/train.txt", encoding='utf8')
trainset = [slowo.replace("\n", "") for slowo in ftrainset.readlines() if len(slowo) < 22]
ftrainset.close()

ftestset = open("../dataset/test.txt", encoding='utf8')
testset = [slowo.replace("\n", "") for slowo in ftestset.readlines()]
ftestset.close()

In [5]:
# trainset = ["abc", "abcd", "abcccc", "bcde"]
# testset = []

In [6]:
print("Train: ", len(trainset))
print("Test: ", len(testset))

Train:  169668
Test:  8931


In [7]:
chartoidx = {}

cnt = 0

longestword = 0

chartoidx["<EMPTY>"] = 0

for slowo in tqdm_notebook(trainset+testset):  
    for litera in slowo:
        if litera not in list(chartoidx.keys()):
            chartoidx[litera] = cnt+1
            cnt = cnt + 1
            
    if len(slowo) > longestword:
        longestword = len(slowo)
            
vocabsize = len(list(chartoidx.keys()))

In [8]:
longestword

19

In [9]:
chartoidx

{'<EMPTY>': 0,
 'p': 1,
 'i': 2,
 'k': 3,
 'u': 4,
 's': 5,
 '8': 6,
 '0': 7,
 'a': 8,
 'n': 9,
 'e': 10,
 'c': 11,
 'A': 12,
 '1': 13,
 '2': 14,
 '3': 15,
 '4': 16,
 '5': 17,
 '6': 18,
 '7': 19,
 '9': 20,
 'l': 21,
 't': 22,
 'g': 23,
 'h': 24,
 'd': 25,
 'm': 26,
 'r': 27,
 'M': 28,
 'I': 29,
 'S': 30,
 'K': 31,
 'J': 32,
 'o': 33,
 'P': 34,
 'j': 35,
 'z': 36,
 'B': 37,
 'U': 38,
 'b': 39,
 'f': 40,
 'w': 41,
 'v': 42,
 'y': 43,
 'H': 44,
 'L': 45,
 'O': 46,
 'T': 47,
 'E': 48,
 'R': 49,
 'N': 50,
 'x': 51,
 'G': 52,
 'q': 53,
 'D': 54,
 'Q': 55,
 'Z': 56,
 'V': 57,
 'C': 58,
 'X': 59,
 'W': 60,
 'Y': 61,
 'F': 62,
 '!': 63,
 '-': 64,
 ' ': 65,
 '@': 66,
 '.': 67,
 '_': 68,
 '/': 69,
 '[': 70,
 ']': 71,
 '~': 72,
 '#': 73,
 '*': 74,
 '(': 75,
 ')': 76,
 '+': 77,
 '`': 78,
 '$': 79,
 ',': 80,
 '%': 81,
 '^': 82,
 '=': 83,
 ';': 84,
 '&': 85,
 '?': 86,
 'ł': 87}

In [10]:
torch.save(chartoidx, "../models/chartoidx.pth")

In [10]:
slowo = trainset[0]; slowo

'pikus80'

In [11]:
window = 5

slowka = ["abc", "defg"]
slowkaidx = [0, 0, 0, 1, 1, 1, 1]

def getitem(slowa, index, window, slowkaidx):
    slowo = slowa[slowkaidx[index]]
    
    idx = slowkaidx[index]
    currindex = 0
    
    while idx == slowkaidx[index]:
        currindex += 1
        idx = slowkaidx[index-currindex]
    currindex -= 1
    
    ins = []
    out = []
    
    ins = [litera for litera in slowo[:currindex+1]]
    for __ in range(window-len(ins)):
        ins.append("<")
    
    if currindex<len(slowo)-1:
        out = [slowo[currindex+1]]
    else:
        out = ["<"]
    
    print(ins, out)

for i in range(len(slowkaidx)):
    getitem(slowka, i, window, slowkaidx)   

['a', '<', '<', '<', '<'] ['b']
['a', 'b', '<', '<', '<'] ['c']
['a', 'b', 'c', '<', '<'] ['<']
['d', '<', '<', '<', '<'] ['e']
['d', 'e', '<', '<', '<'] ['f']
['d', 'e', 'f', '<', '<'] ['g']
['d', 'e', 'f', 'g', '<'] ['<']


In [12]:
class MyDataset(Dataset):
    def __init__(self, slowa, chartoidx, longestword, padding):
        self.slowa = slowa
        self.chartoidx = chartoidx
        self.longestword = longestword
        self.padding = padding
        self.indexpos = []
        
        for idd, slowo in enumerate(slowa):
            for letter in slowo:
                self.indexpos.append(idd)
        
    def __len__(self):
        return len(self.indexpos)
        
    def __getitem__(self, index):
        literyx = []
        literyy = []
        
        slowo = self.slowa[self.indexpos[index]]
        
        
        idx = self.indexpos[index]
        currindex = 0
    
        while idx == self.indexpos[index]:
            currindex += 1
            idx = self.indexpos[index-currindex]
        currindex -= 1
        
        
        literyx = [self.chartoidx[litera] for litera in slowo[:currindex+1]]
        for __ in range(self.longestword-len(literyx)):
            literyx.append(self.chartoidx["<EMPTY>"])
    
        if currindex<len(slowo)-1:
            literyy = [self.chartoidx[slowo[currindex+1]]]
        else:
            literyy = [self.chartoidx["<EMPTY>"]]
                
        return np.array(literyx, dtype="float32"), np.array(literyy, dtype="long")

In [13]:
DS_train = MyDataset(trainset, chartoidx, longestword, 0)
DS_test = MyDataset(testset, chartoidx, longestword, 0)

In [14]:
BS = 10000

In [15]:
DS_train.__getitem__(13)

(array([12.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.], dtype=float32), array([12]))

In [16]:
DL_train = DataLoader(dataset=DS_train, batch_size=BS, num_workers=5)
DL_test = DataLoader(dataset=DS_test, batch_size=BS, num_workers=5)

In [17]:
class CharacterLSTM(nn.Module):
    def __init__(self, vocabsize, lstmlayers, hiddensize):
        super(CharacterLSTM, self).__init__()
        
        
        ## WARSTWY
        self.embd = nn.Embedding(vocabsize, vocabsize)
        self.LSTM1 = nn.GRU(vocabsize, hiddensize, lstmlayers, batch_first=True, bidirectional=True)
        self.linear_ins = nn.Linear(2*hiddensize, vocabsize)
#         self.linear_outs = nn.Linear(2*vocabsize, vocabsize)
        
        ## TRICKY LAYERS
#         self.norm = nn.BatchNorm1d(windowsize)
#         self.norm = nn.LayerNorm(2*vocabsize)
#         self.norm2 = nn.LayerNorm(vocabsize)
        self.drop = nn.Dropout(p=0.1)
        
        ## OUTS
        self.softmax = nn.LogSoftmax(dim=1)
        
        
#         self.init_weights()
        
    def init_weights(self):
        initrange = 0.1
        self.embd.weight.data.uniform_(-initrange, initrange)
        self.linear_ins.bias.data.zero_()
        self.linear_ins.weight.data.uniform_(-initrange, initrange)
#         self.linear_outs.bias.data.zero_()
#         self.linear_outs.weight.data.uniform_(-initrange, initrange)
        
    def forward(self, x, hidden, NLL=True):
#         print(x.shape)
        
        # WEJSCIE
        y0 = self.embd(x)
#         y0  = self.drop(y0)
#         y0 = self.EmbeddingDrop(y0, p=0.2)

        
        # DROP
#         if drop>0:
#             y0 = self.EmbeddingDrop(y0, p=drop)
        
        # LSTM
        y, h1_ = self.LSTM1(y0, hidden)
        
        # TRICKY
#         y = self.norm(y)
        
        y = self.drop(y)
        
        # LINEAR OUT 1
        y = self.linear_ins(y)
#         y = self.norm(y)
#         y = self.linear_ins(y)
        
#         # TRICKY
#         y = self.norm2(y)
    
#         # TRICKY 2
#         y = self.drop(y)
        
        
#         # LINEAR OUT 2
#         y = F.relu(self.linear_outs(y))
      
#         y = self.norm2(y)

#         print(y, y.shape)
        
        if NLL:
            y = self.softmax(y[:,-1])
        
        # zwrot
        return y, h1_
    
#     def EmbeddingDrop(self, dataset, p=0.1):
#         w1, w2, w3 = dataset.shape
#         bern = torch.bernoulli((1-p) * torch.ones((w1, w2, 1))).to("cuda:0")
#         return dataset * bern

In [18]:
lstms = 15
hiddensize = 40

In [19]:
# chlstm = CharacterLSTM(vocabsize, lstms, hiddensize).to(device)
chlstm = torch.load("../models/bezrelu_lstm_15_hidden_40_cosine1e-8_rmsprop1e-7_50epoch_loss_2.408825635910034.pt").to(device)

In [20]:
criterion = nn.BCELoss()
criterionPretraining = nn.CrossEntropyLoss()

In [21]:
epochs = 50

In [22]:
lrmin = 1e-8
lrmax = 1e-2 

def getlr(lrmin, lrmax, epoch, epochs):
    b1 = lrmin
    a1 = (lrmax - b1) / (epochs/2)
    b2 = lrmax - ((-a1)*epochs/2)
    
    lr = 0
    
    if epoch <= epochs/2:
        return  a1*epoch+b1
    else:
        return -a1*epoch+b2

# X1

In [23]:
#BAZA EPOCH x1

onecyclelambda = lambda epoch: getlr(lrmin, lrmax, epoch, epochs)

optimizerLSTM = optim.RMSprop(chlstm.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizerLSTM, epochs, eta_min = 1e-8)
# scheduler = optim.lr_scheduler.LambdaLR(optimizerLSTM, lr_lambda=[onecyclelambda])

In [18]:
t_epochs = tnrange(epochs)

losssess = []

for epoch in t_epochs:
    t_batch = tqdm_notebook(DL_train, leave=False)
    
    for batch in t_batch:
        d1, d2 = batch[0].shape
        xreals = batch[0].long().to(device)
        y = batch[1].long().to(device)
        
        hiddens1 = torch.zeros(2*lstms, d1, hiddensize).to(device)
        
        chlstm.train()
        optimizerLSTM.zero_grad()
        
        y_, _ = chlstm(xreals, hiddens1, NLL=False)
#         print(y_[:,-1], torch.softmax(y_[:,-1], 1))
        loss = criterionPretraining(y_[:,-1], y.view(-1))
        
        loss.backward()
        optimizerLSTM.step()
        
        losss = loss.item()
        losssess.append(losss)
        
        t_batch.set_description("Loss: {:.8f}".format(losss))
        

    t_batch.close()
    t_epochs.set_description("Epoch {}/{}".format(epoch+1, epochs))
    scheduler.step()
    
#     if epoch % 100 == 0:
#         torch.save(chlstm, "../models/bezrelu_lstm_8_hidden_40_bezonecycle_1e-2-1e-8_rmsprop1e-3_700epoch_loss_"+str(losss)+".pt")
    
#     acc = accuracy(chlstm, DL_test, device, lstms, vocabsize)
    print("Epoch {}/{}, Loss {:.8f}, LR {:.8f}".format(epoch+1, epochs, losss, scheduler.get_lr()[0]))
#     print("Epoch {}/{}, Loss {:.8f}".format(epoch+1, epochs, losss))

NameError: name 'epochs' is not defined

In [25]:
sum(p.numel() for p in chlstm.parameters())

455992

In [26]:
torch.save(losssess, "../wykresy/bezrelu_lstm_15_hidden_40_cosine1e-8_rmsprop1e-7_50epoch_loss_"+str(losss)+".pt")

In [27]:
torch.save(chlstm, "../models/bezrelu_lstm_15_hidden_40_cosine1e-8_rmsprop1e-7_50epoch_loss_"+str(losss)+".pt")

/home/mborzymowski/.conda/envs/mborzymowski/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type CharacterLSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [25]:
torch.save(chlstm, "../models/bezrelu_500epoch_onecycle_1e-2-1e-8_rmsprop1e-3_4lstm_50hidd.pt")

/home/mborzymowski/.conda/envs/mborzymowski/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type CharacterLSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [35]:
# torch.save(losssess, "../wykresy/600epoch_onecycle_1e-2-1e-6_rmsprop1e-3_4lstm_50hidd.pt")

In [22]:
losssess = torch.load("../wykresy/600epoch_onecycle_1e-2-1e-6_rmsprop1e-3_4lstm_50hidd.pt")

In [23]:
%matplotlib notebook
fix, ax = plt.subplots()
ax.plot([x for x in range(len(losssess))], losssess, ".")

FigureCanvasNbAgg()

In [81]:
str(["a" for i in range(3)])

"['a', 'a', 'a']"

In [23]:
chardict = list(chartoidx.keys())
temperature = 0.35

# plikhasel = open("../haslaAI_bezrelu.txt", "a", encoding="utf8")

hiddens = torch.zeros(2*lstms, 1, hiddensize).to(device)

for _ in tqdm_notebook(range(20)):
    slowostart = "d"

    with torch.no_grad():
        lastchar = 0
        cnt = 0

        chlstm.eval()
        znaki = [chartoidx[item] for item in slowostart]
        bazastart = len(znaki)

        for __ in range(longestword - bazastart+1):
            znaki.append(chartoidx["<EMPTY>"])

        for item in range(longestword - bazastart+1):
            x = torch.Tensor(znaki).long().to(device).view(1, -1)

            out, _ = chlstm(x, hiddens, NLL=False)

            print(x.shape)
            print(out.shape)
            
            zwrot = out[:,-1].view(-1)
            exped = zwrot.data.div(temperature).exp()
            top_i = torch.multinomial(exped, 1)
    #         softmaxed = torch.softmax(top_i, 0)
    #         charid = torch.argmax(softmaxed, 0)
            charid = top_i[0]

            znaki[item+bazastart] = charid


        slowo = ""
        for item in znaki:
            if item != 0:
                slowo+=chardict[item]

        print(slowo)
#         plikhasel.write(slowo+"\n")
# plikhasel.close()

torch.Size([1, 20])
torch.Size([1, 20, 88])
torch.Size([1, 20])
torch.Size([1, 20, 88])
torch.Size([1, 20])
torch.Size([1, 20, 88])
torch.Size([1, 20])
torch.Size([1, 20, 88])
torch.Size([1, 20])
torch.Size([1, 20, 88])
torch.Size([1, 20])
torch.Size([1, 20, 88])
torch.Size([1, 20])
torch.Size([1, 20, 88])
torch.Size([1, 20])
torch.Size([1, 20, 88])
torch.Size([1, 20])
torch.Size([1, 20, 88])
torch.Size([1, 20])
torch.Size([1, 20, 88])
torch.Size([1, 20])
torch.Size([1, 20, 88])
torch.Size([1, 20])
torch.Size([1, 20, 88])
torch.Size([1, 20])
torch.Size([1, 20, 88])
torch.Size([1, 20])
torch.Size([1, 20, 88])
torch.Size([1, 20])
torch.Size([1, 20, 88])
torch.Size([1, 20])
torch.Size([1, 20, 88])
torch.Size([1, 20])
torch.Size([1, 20, 88])
torch.Size([1, 20])
torch.Size([1, 20, 88])
torch.Size([1, 20])
torch.Size([1, 20, 88])
danio2001
torch.Size([1, 20])
torch.Size([1, 20, 88])
torch.Size([1, 20])
torch.Size([1, 20, 88])
torch.Size([1, 20])
torch.Size([1, 20, 88])
torch.Size([1, 20])
to

KeyboardInterrupt: 

In [16]:
torch.save([longestword, lstms, hiddensize], "../models/longestword.pth")

In [79]:
chardict = list(chartoidx.keys())
temperature = 0.5
ilehasel = 100
startrange = 3

fakehasla = []
startowegen = list(itertools.permutations(chardict[1:62], startrange))
startowe = []

for it in startowegen:
    slw = ""
    for letter in it:
        slw+=letter
    startowe.append(slw)
    
    
plikhasel = open("../haslaAI_bezrelu.txt", "a", encoding="utf8")

for start in tqdm_notebook(startowe):
    hiddens = torch.zeros(2*lstms, ilehasel, hiddensize).to(device)
    slowostart = start
    
    with torch.no_grad():
        lastchar = 0
        cnt = 0

        chlstm.eval()
        znaki = [[chartoidx[item] for item in slowostart] for _ in range(ilehasel)]
        bazastart = len(znaki[0])

        for i in range(ilehasel):
            for __ in range(longestword - bazastart+1):
                znaki[i].append(chartoidx["<EMPTY>"])

        for item in range(longestword - bazastart+1):
            x = torch.Tensor(znaki).long().to(device).view(ilehasel, -1)

            out, hiddens = chlstm(x, hiddens, NLL=False)

            zwrot = out[:,-1].view(ilehasel, -1)
            exped = zwrot.data.div(temperature).exp()
            top_i = torch.multinomial(exped, 1)
    #         softmaxed = torch.softmax(top_i, 0)
    #         charid = torch.argmax(softmaxed, 0)

            for i in range(len(top_i)):
                charid = top_i[i][0]
                znaki[i][item+bazastart] = charid

        for haslo in znaki:
            slowo = ""
            for item in haslo:
                if item != 0:
                    slowo+=chardict[item]
            plikhasel.write(slowo+"\n")
plikhasel.close()

KeyboardInterrupt: 